[ ] Чаще покупаю на повышение средней цены или понижение?

In [ ]:
import datetime as dt
from pathlib import Path
from functools import reduce
import math
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from matplotlib.dates import MonthLocator

In [ ]:
month_locator = MonthLocator()

# Загрузка данных

## Операции Тинькофф

In [ ]:
operations = pd.read_csv('data/tinkoff/operations.csv', index_col='id', parse_dates=['dt']).sort_values(by='dt')
operations['total_price'] = operations['total_price'].apply(lambda x: -x)
operations['count'] = operations.apply(lambda x: -x['count'] if x['operation_type'] == 'sell' else x['count'], axis=1)
operations = operations[operations['operation_type'].isin(['buy', 'sell', 'buy_card'])]
# operations = operations.groupby('ticker', as_index=False).first()

buy_list = operations[['dt', 'isin', 'ticker', 'operation_type', 'count', 'unit_price', 'total_price']]
buy_list['date'] = buy_list['dt'].dt.date
buy_list['cum_count'] = buy_list.groupby('ticker')['count'].cumsum()
buy_list['cum_spent'] = buy_list.groupby('ticker')['total_price'].cumsum()
buy_list['avg_price'] = buy_list.apply(lambda x: None if x['cum_count'] == 0 else x['cum_spent'] / x['cum_count'], axis=1)
buy_list.head()

In [ ]:
ticker_cumcount = buy_list.groupby('ticker').last()['cum_count'] > 0
active_ticker = ticker_cumcount[ticker_cumcount].index
buy_list = buy_list[buy_list['ticker'].isin(active_ticker)]

In [ ]:
# count_l = buy_list['count'].apply(lambda x: [] if pd.isnull(x) else [int(x)])
# cum_list_count = count_l.groupby(buy_list['ticker']).apply(lambda x: x.cumsum())

In [ ]:
# spent_l = buy_list['total_price'].apply(lambda x: [] if pd.isnull(x) else [x])
# cum_list_spent = spent_l.groupby(buy_list['ticker']).apply(lambda x: x.cumsum())

In [ ]:
# buy_list['list_buy'] = [
#     [(x, y) for x, y in zip(*zip_first)]
#     for zip_first in list(zip(cum_list_count, cum_list_spent))
# ]

In [ ]:
stocks = []
for path in Path('data/investfunds/').iterdir():
    if not path.suffix == '.csv':
        continue
    df = (pd.read_csv(path, parse_dates=['dt'])
          .rename(columns={'dt': 'date'})
          .sort_values(by='date')
          .reset_index(drop=True)
          .assign(isin=path.stem))
    df['date'] = df['date'].dt.date
    stocks.append(df)
stocks = pd.concat(stocks, axis=0)

## Подсчет итоговых данных

In [ ]:
merged_list = stocks.merge(buy_list[['date', 'isin', 'ticker', 'operation_type', 'cum_count', 'cum_spent', 'avg_price']], on=['date', 'isin'], how='left')
merged_list.loc[:, ['ticker', 'cum_count', 'cum_spent', 'avg_price']] = merged_list.groupby('isin').ffill()[['ticker', 'cum_count', 'cum_spent', 'avg_price']]
# подсчет итоговой прибыли в абсолютном значении
merged_list['profit'] = merged_list.apply(lambda x: x['cum_count'] * (x['close_price'] - x['avg_price']), axis=1)
# подсчет итоговой прибыли в процентах
merged_list['profit_percent'] = merged_list.apply(lambda x: x['cum_count'] * (x['close_price'] - x['avg_price']) / x['cum_spent'], axis=1).mul(100)
# удаляем записи, где отсутствует общее количество акций
merged_list.loc[merged_list['cum_count'] == 0, 'avg_price'] = None
merged_list = merged_list.dropna(subset=['cum_count', 'avg_price'])

In [ ]:
# процент ETF по типам активов
last_day = merged_list[merged_list['date'] == merged_list['date'].max()]
last_day.groupby('investemnt_object_type').apply(lambda x: round(x['cum_spent'].sum() / last_day['cum_spent'].sum(), 2)).mul(100)

# График общей прибыли

In [ ]:
buy_date = buy_list.copy().set_index('date')
buy_date = buy_date['cum_spent'] - buy_date['cum_spent'].shift()
buy_date = buy_date[buy_date > 0]
buy_date = buy_date.groupby('date').sum().sort_index().rename('sum').reset_index()
buy_date.head()

In [ ]:
all_profit = merged_list.groupby('date').apply(lambda x: (x['cum_count'] * (x['close_price'] - x['avg_price'])).sum() / x['cum_spent'].sum()).mul(100).rename('profit').reset_index()
all_profit = all_profit.merge(buy_date, on='date', how='left')
all_profit['profit'].agg(
    min_profit='min',
    max_profit='max',
    last_profit=lambda x: x.iloc[-1],
).to_frame().T

In [ ]:
buy_date_size_point = all_profit.dropna()
buy_date_size_point = buy_date_size_point['sum'].div(buy_date_size_point['sum'].max()).mul(100).values

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.lineplot(data=all_profit, x='date', y='profit')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.7)
plt.scatter(all_profit.dropna()['date'], all_profit.dropna()['profit'], s=buy_date_size_point, c='r')
plt.grid(linestyle='--', alpha=0.3)
plt.title('График прибыльности во времени')
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.gca().xaxis.set_major_locator(month_locator)
plt.savefig('artifacts/profit.png')

# Прибыль по типам бумаг

In [ ]:
type_profit = merged_list.groupby(['date', 'investemnt_object_type']).apply(lambda x: (x['cum_count'] * (x['close_price'] - x['avg_price'])).sum() / x['cum_spent'].sum()).mul(100).rename('profit').reset_index()
type_profit.groupby('investemnt_object_type').agg(
    min_profit=('profit', 'min'),
    max_profit=('profit', 'max'),
    last_profit=('profit', 'last'),
    days=('date', lambda x: x.max() - x.min())
)

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.lineplot(data=type_profit, x='date', y='profit', hue='investemnt_object_type')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.7)
plt.grid(linestyle='--', alpha=0.3)
plt.title('График прибыльности по типам')
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.gca().xaxis.set_major_locator(month_locator)
plt.savefig('artifacts/type_profit.png')

## Корреляции между типами

In [ ]:
gold = type_profit[(type_profit['date'] >= dt.date(2020, 11, 1)) & (type_profit['investemnt_object_type'] == 'Драгметаллы')]
stock = type_profit[(type_profit['date'].isin(gold['date'].unique())) & (type_profit['investemnt_object_type'] == 'Акции')]
bond = type_profit[(type_profit['date'].isin(gold['date'].unique())) & (type_profit['investemnt_object_type'] == 'Облигации')]

print('Акции vs Драгметаллы:', round(np.corrcoef(stock['profit'], gold['profit'])[0][1], 3))
print('Акции vs Облигации:', round(np.corrcoef(stock['profit'], bond['profit'])[0][1], 3))
print('Драгметаллы vs Облигации:', round(np.corrcoef(gold['profit'], bond['profit'])[0][1], 3))

# График прибыли по тикерам

In [ ]:
merged_list.groupby('ticker').agg(
    cnt=('cum_count', 'last'),
    avg_price=('avg_price', 'last'),
    min_profit=('profit_percent', 'min'),
    max_profit=('profit_percent', 'max'),
    last_profit=('profit_percent', 'last'),
    days=('date', lambda x: x.max() - x.min())
).sort_values(by='max_profit', ascending=False)

In [ ]:
tickers = merged_list['ticker'].unique()
total_tickers = len(tickers)
max_cols = 2
max_rows = math.ceil(total_tickers / max_cols)
fig = plt.figure(figsize=(10 * max_cols, 5 * max_rows)) 
for index, ticker in enumerate(tickers[:total_tickers], start=1):
    ticker_data = merged_list[merged_list['ticker'] == ticker]
    ax = plt.subplot(max_rows, max_cols, index)
    ax.set_xlabel('Дата')
    ax.set_ylabel('Цена, руб.')
    l1 = ax.plot(merged_list.loc[merged_list['ticker'] == ticker, 'date'], merged_list.loc[merged_list['ticker'] == ticker, 'close_price'], color='blue', label='Цена')

    ax2 = ax.twinx()
    ax2.set_ylabel('Доходность, %')
    l2 = ax2.plot(merged_list.loc[merged_list['ticker'] == ticker, 'date'], merged_list.loc[merged_list['ticker'] == ticker, 'profit_percent'], color='red', label='Прибыль')
    ax.grid(linestyle='--', alpha=0.5)
    plt.title(f'{ticker}')
    
    for label in ax.get_xticklabels():
        label.set_ha("right")
        label.set_rotation(30)
    lns = l1+l2
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc=0)
    ax.xaxis.set_major_locator(month_locator)
    
plt.tight_layout()
plt.savefig('artifacts/price_vs_profit.png')

# Корреляции

In [ ]:
dt_price = merged_list[['date', 'ticker', 'close_price']].drop_duplicates()
dt_price = dt_price.set_index(['date', 'ticker']).unstack()
dt_price.columns = [c[1] for c in dt_price.columns]
dt_price = dt_price.dropna()

corr = dt_price.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.color_palette('coolwarm', as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, vmin=-1, annot=True, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.savefig('artifacts/corr_matrix.png')

# График прибыли по странам (все бумаги страны)

In [ ]:
country_type_profit = merged_list.groupby(['date', 'investemnt_object_type', 'geography']).apply(lambda x: (x['cum_count'] * (x['close_price'] - x['avg_price'])).sum() / x['cum_spent'].sum()).mul(100).rename('profit').reset_index()

fig = plt.figure(figsize=(20, 10))
sns.lineplot(data=country_type_profit[country_type_profit['investemnt_object_type'] == 'Акции'], x='date', y='profit', hue='geography')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.7)
plt.grid(linestyle='--', alpha=0.3)
plt.title('График прибыльности акций по странам')
plt.tight_layout()
plt.gca().xaxis.set_major_locator(month_locator)
plt.savefig('artifacts/country_stock_profit.png')

fig = plt.figure(figsize=(20, 10))
sns.lineplot(data=country_type_profit[country_type_profit['investemnt_object_type'] == 'Облигации'], x='date', y='profit', hue='geography')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.7)
plt.grid(linestyle='--', alpha=0.3)
plt.title('График прибыльности облигаций по странам')
plt.tight_layout()
plt.gca().xaxis.set_major_locator(month_locator)
plt.savefig('artifacts/country_bond_profit.png')